# Dividend & Company Foundation Analysis
This project is to study and analyze what makes a company potentially good for investment. There are a few financial metrics/ratios where its value can be analyzed that tells if the company is healthy or growing. The companies that meet the requirements from certain ranges of these values can be used as an indicative are specified below:

### Foundational Ratios
1) Earnings Per Share  >= 10%


2) Price to Earnings - <= 10%


3) Operating Margins - >= 10%


4) Return of Equity - >= 20%


5) Free Cash Flow - >= 0


### Dividend Ratios
1) Annual Dividend Yield - >= 2%


2) Payout Ratio - >= 25% && <= 75%


The project first reads the list of stock symbols (provided by Bursa Marketplace) that can be used as URL or reference to retrieve these metrics/ratios. By means of yfinance API and intelligent web scraping, these values can be easily retrieved by demonstrating an intelligent looping process that iterates the list of stock symbols and running the processes accordingly. 

In [1]:
# import libraries
import datetime
from pandas_datareader import data as pdr
import yfinance as yf
import pandas as pd
import time
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import random
import warnings
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 10)

# Part I: Retrieving the Dividend & Foundation Ratios (yfinance API & Web Scraping)

## Reading the List of Stock Symbols
The stocks.csv is read to gather the symbols as idenfication for web scraping and API argument to get its respective values.

In [2]:
df = pd.read_csv('exports/stocks.csv')
df

,stock_name,stock_code
0,7-ELEVEN MALAYSIA HOLDINGS BERHAD,5250
1,ABF MALAYSIA BOND INDEX FUND,0800EA
2,ABLE GLOBAL BERHAD,7167
3,ABLEGROUP BERHAD,7086
4,ABM FUJIYA BERHAD,5198
...,...,...
1008,YKGI HOLDINGS BERHAD,7020
1009,YLI HOLDINGS BERHAD,7014
1010,YNH PROPERTY BERHAD,3158
1011,YONG TAI BERHAD,7066


## .KL is added to support yfinance argument format to reference the stock symbol

In [3]:
# Add ".KL" to each value in the 'stock_code' column
testdf = df['stock_code'].apply(lambda x: x + ".KL")
testdf

0         5250.KL
1       0800EA.KL
2         7167.KL
3         7086.KL
4         5198.KL
          ...    
1008      7020.KL
1009      7014.KL
1010      3158.KL
1011      7066.KL
1012      5159.KL
Name: stock_code, Length: 1013, dtype: object

In [4]:
# Put it into a list called stock_code
stock_code = list(testdf)
stock_code

['5250.KL',
 '0800EA.KL',
 '7167.KL',
 '7086.KL',
 '5198.KL',
 '03028.KL',
 '7131.KL',
 '0218.KL',
 '0122.KL',
 '1481.KL',
 '5281.KL',
 '9148.KL',
 '7191.KL',
 '7146.KL',
 '0181.KL',
 '6599.KL',
 '5139.KL',
 '5185.KL',
 '7145.KL',
 '0258.KL',
 '7315.KL',
 '7078.KL',
 '0209.KL',
 '5238.KL',
 '2658.KL',
 '7609.KL',
 '5116.KL',
 '5115.KL',
 '2674.KL',
 '0079.KL',
 '2488.KL',
 '1163.KL',
 '03051.KL',
 '5269.KL',
 '5127.KL',
 '5293.KL',
 '5307.KL',
 '5120.KL',
 '03011.KL',
 '1015.KL',
 '7031.KL',
 '6351.KL',
 '7083.KL',
 '0048.KL',
 '4758.KL',
 '0226.KL',
 '6556.KL',
 '5082.KL',
 '5568.KL',
 '5088.KL',
 '7090.KL',
 '5015.KL',
 '6432.KL',
 '0119.KL',
 '7214.KL',
 '7181.KL',
 '7007.KL',
 '0038.KL',
 '0068.KL',
 '7722.KL',
 '7129.KL',
 '0159.KL',
 '0105.KL',
 '4057.KL',
 '7162.KL',
 '7054.KL',
 '03032.KL',
 '6399.KL',
 '0072.KL',
 '8176.KL',
 '7048.KL',
 '5130.KL',
 '7099.KL',
 '5302.KL',
 '03037.KL',
 '8885.KL',
 '5204.KL',
 '7579.KL',
 '6888.KL',
 '5106.KL',
 '7120.KL',
 '2305.KL',
 '5021.KL

## Getting the Dividend and Financial Ratios of the listed companies
The code below demonstrates retrieving the important dividend and financial ratios of the listed companies from their symbols. The values are being sourced from Yahoo Finance and Bursa Marketplace to ensure data integrity and consistency. Dividend Yield, Payout Ratio and Operating Margins are retrieved through Yahoo Finance with yfinance API. Price to Earnings, Return of Equity and Operating Margins are retrieved from Bursa Marketplace through a complex method of web scraping.


In [7]:
driver = webdriver.Chrome()
driver.implicitly_wait(20)

# funtion to get link
def get_link(stock_code):
    
    url='https://www.bursamarketplace.com/index.php'
    driver.get(url)
    
    # Clicking the Search Button
    s= driver.find_element(By.XPATH,'//*[@id="newnav-mobileSearch"]')
    driver.execute_script("arguments[0].click();",s)
    
    # Inputting the Stock Code in the Search Bar
    driver.find_element(By.XPATH, '//*[@id="newnav-search-input"]').send_keys(stock_code)
    
    # Getting the Link in the appeared Search Results
    lnks = driver.find_element(By.ID,'url')
    links=lnks.get_attribute('href')
    return links

In [8]:
results_df = []
new_list = []

# Define the columns you want to retrieve
columns_to_fetch = ["dividendYield", "payoutRatio", "operatingMargins"]

# Create an empty DataFrame to store the results
results_df = pd.DataFrame(columns=['Ticker'] + columns_to_fetch)

# Loop through each ticker in the CSV
for ticker_symbol in stock_code:
    try:
        # Create a Ticker object using yfinance
        ticker = yf.Ticker(ticker_symbol)

        # Get the info for the ticker
        ticker_info = ticker.info

        # Extract the desired columns from the ticker info
        row_data = {
            'stock_code': ticker_symbol,
        }

        for column in columns_to_fetch:
            if column in ticker_info:
                row_data[column] = ticker_info[column]
            else:
                row_data[column] = None
        print(row_data)
        
        # Get the cashflow data
        cashflow_df = ticker.cashflow
        
        # Extract the latest Free Cash Flow value dynamically
        latest_date = cashflow_df.columns[0]  # Get the last column (latest date)
        row_data['CF'] = cashflow_df.loc['Free Cash Flow', latest_date]
        
        url=get_link(ticker_symbol[:-3]) # Exclude ".KL for Bursa Marketplace Search Function"
        driver.get(url)
        content = driver.page_source
        soup = bs(content)
        
        time.sleep(5)
        
        try:
            stock_PE = driver.find_element(By.XPATH, "//div[text()='P/E']/following-sibling::div")
            row_data['P/E'] = stock_PE.text
            stock_ROE = driver.find_element(By.XPATH, "//div[text()='ROE']/following-sibling::div")
            row_data['ROE'] = stock_ROE.text
            stock_EPS = driver.find_element(By.XPATH, "//div[text()='EPS']/following-sibling::div")
            row_data['EPS'] = stock_EPS.text
        except:
             print('stock code :'+s+' cannot scrape values')

        # Append the data as a row to the results DataFrame
        results_df = pd.concat([results_df, pd.DataFrame([row_data])], ignore_index=True)
        

    except Exception as e:
        print(f"Error fetching data for {ticker_symbol}: {str(e)}")
        new_list.append(ticker_symbol)
    




SyntaxError: f-string: closing parenthesis '}' does not match opening parenthesis '(' (4013597083.py, line 53)

In [ ]:
# Save the results to a CSV file
results_df.to_csv('exports/stock_info.csv', index=False)

# Display the results DataFrame
print(results_df)

## There seems to be Symbols that have faced some errors in retrieving data
These symbols are separated in another list variable showed below (and saved to a separate CSV)

In [16]:
dflist = pd.DataFrame(new_list, columns=['stock_code'])

# Specify the file path where you want to save the CSV file
csv_file_path = 'exports/errorprices.csv'

# Use the to_csv() method to export the DataFrame to a CSV file
dflist.to_csv(csv_file_path, index=False)  # Set index=False to exclude the DataFrame index from the CSV

print(f"DataFrame saved to {csv_file_path}")

DataFrame saved to exports/errorprices.csv


In [13]:
new_list

['0800EA.KL',
 '2771.KL',
 '3484.KL',
 '0400GB.KL',
 '5835.KL',
 '03055.KL',
 '0820EA.KL',
 '7192.KL',
 '5108.KL',
 '0401GA.KL',
 '8931.KL',
 '0835EA.KL',
 '0834EA.KL',
 '6645.KL',
 '0821EA.KL',
 '0827EA.KL',
 '0824EA.KL',
 '0825EA.KL',
 '0822EA.KL',
 '0823EA.KL',
 '5270.KL',
 '7158.KL',
 '5268.KL',
 '0836EA.KL',
 '0833EA.KL',
 '0832EA.KL',
 '0837EA.KL',
 '0831EA.KL',
 '0830EA.KL',
 '0829EA.KL',
 '0829EB.KL',
 '0828EA.KL',
 '0838EA.KL']

## Another loop was carried out that iterates the error symbols in an attempt to retrieve the values again
Just to ensure if some errors are temporary, the list of error symbols are iterated with the repeated process. The same results occur.

In [33]:
driver = webdriver.Chrome()
driver.implicitly_wait(20)

# funtion to get link
def get_link2(stock_code):
    
    url='https://www.bursamarketplace.com/index.php'
    driver.get(url)
    
    #click search button
    s= driver.find_element(By.XPATH,'//*[@id="newnav-mobileSearch"]')
    driver.execute_script("arguments[0].click();",s)
    
    # input code
    driver.find_element(By.XPATH, '//*[@id="newnav-search-input"]').send_keys(stock_code)
    
    # get the link
    lnks = driver.find_element(By.ID,'url')
    links=lnks.get_attribute('href')
    return links

# Define the columns you want to retrieve
columns_to_fetch = ["dividendYield", "payoutRatio", "operatingMargins"]

# Create an empty DataFrame to store the results
results_df = pd.DataFrame(columns=['Ticker'] + columns_to_fetch)

# Loop through each ticker in the CSV
for ticker_symbol in new_list:
    try:
        # Create a Ticker object using yfinance
        ticker = yf.Ticker(ticker_symbol)

        # Get the info for the ticker
        ticker_info = ticker.info

        # Extract the desired columns from the ticker info
        row_data = {
            'Ticker': ticker_symbol,
        }

        for column in columns_to_fetch:
            if column in ticker_info:
                row_data[column] = ticker_info[column]
            else:
                row_data[column] = None
        print(row_data)
        
        # Get the cashflow data
        cashflow_df = ticker.cashflow
        
        # Extract the latest Free Cash Flow value dynamically
        latest_date = cashflow_df.columns[0]  # Get the last column (latest date)
        row_data['CF'] = cashflow_df.loc['Free Cash Flow', latest_date]
        
        url=get_link2(ticker_symbol[:-3]) # Exclude ".KL for Bursa Marketplace Search Function"
        driver.get(url)
        content = driver.page_source
        soup = bs(content)
        
        time.sleep(5)
        
        try:
            stock_PE = driver.find_element(By.XPATH, "//div[text()='P/E']/following-sibling::div")
            row_data['P/E'] = stock_PE.text
            stock_ROE = driver.find_element(By.XPATH, "//div[text()='ROE']/following-sibling::div")
            row_data['ROE'] = stock_ROE.text
            stock_EPS = driver.find_element(By.XPATH, "//div[text()='EPS']/following-sibling::div")
            row_data['EPS'] = stock_EPS.text
        except:
             print('stock code :'+s+' cannot scrape values')

        # Append the data as a row to the results DataFrame
        results_df = pd.concat([results_df, pd.DataFrame([row_data])], ignore_index=True)
        

    except Exception as e:
        print(f"Error fetching data for {ticker_symbol}: {str(e)}")

{'Ticker': '0800EA.KL', 'dividendYield': None, 'payoutRatio': None, 'operatingMargins': None}
Error fetching data for 0800EA.KL: index 0 is out of bounds for axis 0 with size 0
{'Ticker': '2771.KL', 'dividendYield': None, 'payoutRatio': None, 'operatingMargins': None}
Error fetching data for 2771.KL: index 0 is out of bounds for axis 0 with size 0
{'Ticker': '3484.KL', 'dividendYield': None, 'payoutRatio': None, 'operatingMargins': None}
Error fetching data for 3484.KL: index 0 is out of bounds for axis 0 with size 0
{'Ticker': '0400GB.KL', 'dividendYield': None, 'payoutRatio': None, 'operatingMargins': None}
Error fetching data for 0400GB.KL: index 0 is out of bounds for axis 0 with size 0
{'Ticker': '5835.KL', 'dividendYield': None, 'payoutRatio': None, 'operatingMargins': None}
Error fetching data for 5835.KL: index 0 is out of bounds for axis 0 with size 0
{'Ticker': '03055.KL', 'dividendYield': None, 'payoutRatio': None, 'operatingMargins': 0.0}
Error fetching data for 03055.KL: i

## Therefore, the resulting dataframe can be proceed excluding these error Tickers
The tickers that failed to retrieve the values are due to various reasons that cannot be addressed at this current time:
1) The search engine in Bursa Malaysia did not manage to render the link to the ticker from its search function. This is probably due to the company recently being delisted in Bursa Marketplace sometime after the list of stocks was released.


2) Unable to resolve the HTML element in Bursa Marketplace ticker page. One or more elements that displays the required ratios could not be found on some symbols after loading its respective link in Bursa Marketplace. These symbols are excluded for incomplete information of potentially healthy companies.


2) Could not resolve cash flow statement from Yahoo Finance. Free cash flow is an important metric that was calculated by Yahoo Finance through their cash flow reports. These symbols are excluded for incomplete information for filtration of potentially healthy companies.

In [41]:
new_list

['5082.KL',
 '6602.KL',
 '0400GB.KL',
 '2143.KL',
 '0116.KL',
 '5020.KL',
 '9113.KL',
 '8931.KL',
 '7096.KL',
 '7153.KL',
 '8303.KL',
 '5001.KL',
 '0035.KL',
 '7055.KL',
 '0260.KL',
 '5270.KL',
 '5285.KL',
 '5140.KL',
 '8931.KL',
 '5270.KL',
 '8931.KL',
 '5270.KL',
 '8931.KL',
 '5270.KL',
 '8931.KL',
 '5270.KL',
 '8931.KL',
 '5270.KL',
 '8931.KL',
 '5270.KL',
 '8931.KL',
 '5270.KL',
 '8931.KL',
 '5270.KL',
 '8931.KL',
 '5270.KL',
 '8931.KL',
 '5270.KL',
 '8931.KL',
 '5270.KL',
 '8931.KL',
 '5270.KL',
 '8931.KL',
 '5270.KL',
 '8931.KL',
 '5270.KL',
 '8931.KL',
 '5270.KL',
 '8931.KL',
 '5270.KL',
 '8931.KL',
 '5270.KL',
 '8931.KL',
 '5270.KL',
 '8931.KL',
 '5270.KL',
 '8931.KL',
 '5270.KL',
 '8931.KL',
 '5270.KL',
 '8931.KL',
 '5270.KL',
 '8931.KL',
 '5270.KL',
 '8931.KL',
 '5270.KL',
 '8931.KL',
 '5270.KL',
 '8931.KL',
 '5270.KL',
 '8931.KL',
 '5270.KL',
 '8931.KL',
 '5270.KL',
 '8931.KL',
 '5270.KL',
 '8931.KL',
 '5270.KL',
 '8931.KL',
 '5270.KL',
 '8931.KL',
 '5270.KL',
 '8931.KL',
 '

## Successfully retrieved the Dividend & Foundational Ratios of the list of Stock Symbols. With the resulting Dataframe, the retrieved data are saved in a CSV file as a checkpoint

In [42]:
# Save the results to a CSV file
results_df.to_csv('stock_info.csv', index=False)

In [3]:
df2 = pd.read_csv('exports/stock_info.csv')
df2

,Ticker,dividendYield,payoutRatio,operatingMargins,CF,P/E,ROE,EPS
0,5250.KL,0.0271,1.0385,0.04819,353993000.0,38.19,51.103,0.060
1,7167.KL,0.0465,0.4039,0.08492,-143040391.0,12.02,8.475,0.109
2,7086.KL,NaN,0.0000,0.04338,810389.0,108.11,0.674,0.000
3,5198.KL,NaN,0.0000,0.01950,-45692900.0,-,-3.447,-0.012
4,03028.KL,0.0566,99.0000,-0.00005,-5502239.0,-,-0.253,0.000
...,...,...,...,...,...,...,...,...
975,7020.KL,NaN,0.0000,-0.00534,3660971.0,-,-11.211,-0.013
976,7014.KL,NaN,0.0000,0.08203,-8095000.0,7.74,3.636,0.033
977,3158.KL,NaN,0.0000,0.23088,24588648.0,-,-2.587,-0.005
978,7066.KL,NaN,0.0000,-0.01471,-48212179.0,-,-8.419,-0.060


# Part II: Data Cleaning & Symbol Selection
Now that the ratios are retrieved for the stock symbols, additional preprocessing is carried out to prepare the data before selecting the potentially healthy and growing companies. 

## Imputing Missing or "-" values
Some of these retrieved values are displayed as "-" in Bursa Marketplace or Yahoo Finance which indicate incomplete information. These values are imputed as 0.

In [4]:
import pandas as pd

# Impute missing values in 'dividendYield' and 'payoutRatio' columns with 0
df2['dividendYield'].fillna(0, inplace=True)
df2['payoutRatio'].fillna(0, inplace=True)
df2['operatingMargins'].fillna(0, inplace=True)
# Impute dashes in 'P/E' column with 0
df2['P/E'].replace('-', 0, inplace=True)
df2['ROE'].replace('-', 0, inplace=True)
df2['EPS'].replace('-', 0, inplace=True)

# Save the DataFrame back to a CSV file
df2.to_csv('exports/imputed_file.csv', index=False)


## Converting Data Types
The data types are converted from string to float to carry out conditional processes to assign classes.

In [5]:
# Convert specific columns to float
columns_to_convert = ['dividendYield', 'payoutRatio', 'operatingMargins', 'CF', 'P/E', 'ROE', 'EPS']

for column in columns_to_convert:
    df2[column] = pd.to_numeric(df2[column], errors='coerce').fillna(0)

In [6]:
# # Define the filtering conditions
# conditions = (df2['dividendYield'] >= 0.02) & \
#               (df2['payoutRatio'].between(0.2, 0.75)) & \
#               (df2['operatingMargins'] >= 0.1) & \
#               (df2['CF'] >= 0) & \
#               (df2['P/E'] <= 10) & \
#               (df2['ROE'] >= 20) & \
#               (df2['EPS'] >= 0.1)

# # Apply the conditions to filter the DataFrame
# filtered_df = df2[conditions]

# # Save the filtered DataFrame to a new CSV file
# filtered_df.to_csv('filtered_csv_file.csv', index=False)  # Replace 'filtered_csv_file.csv' with the desired output file path

## Assigning Classes to Stock Symbols based on Ratio Values

Now we can proceed to categorize the stock symbols in classes based on values of their retrieved ratios. Firstly, we also have to categorize the ratios based on its relation towards Dividends, or the company's Foundational Health:

Foundation Ratios: Operating Margins, Price to Earnings, Return of Equity, Earnings Per Share and Free Cash Flow
Dividend Ratios: Dividend Yield, Payout Ratio


The classes defined are Class A, B, C, D which are assigned based on the conditions on the above categories:

1) Class A - Foundation and Dividend meets requirements


2) Class B - Foundation meets requirements, Dividend does not meet requirements


3) Class C - Foundation does not meet requirements, Dividend meet requirements


4) Class D - Foundation and Dividend do not meet requirements

In [7]:
import pandas as pd
import numpy as np

# Define the DataFrame 'df2' with your data

# Define the filtering conditions
dividend_condition = (df2['dividendYield'] >= 0.02) & (df2['payoutRatio'].between(0.1, 0.75))
foundation_condition = (df2['operatingMargins'] >= 0.1) & (df2['CF'] >= 0) & (df2['P/E'] <= 10) & (df2['ROE'] >= 20) & (df2['EPS'] >= 0.1)

# Define the choices and values for the 'class' column
choices = [
    np.select([dividend_condition, foundation_condition], [1, 1]),  # Class A
    np.select([dividend_condition, foundation_condition], [0, 1]),  # Class B
    np.select([dividend_condition, foundation_condition], [1, 0]),  # Class C
    np.select([dividend_condition, foundation_condition], [0, 0])   # Class D
]

# Create the 'class' column based on the choices
df2['class'] = np.select([dividend_condition & foundation_condition, foundation_condition, dividend_condition], ['A', 'B', 'C'], default='D')

# Save the DataFrame to a new CSV file
df2.to_csv('stock_class.csv', index=False)  # Replace 'output_csv_file.csv' with the desired output file path


## Selecting the companies that have good investment potential
With the approach stated above, we can identify which companies are performing well based on our investment requirements as well as whether or not they provide dividends. Based on the foundation and dividend, our strategy will favor companies that fall into Classes A or B. 

In [24]:
import pandas as pd

# Load the filtered DataFrame (assuming you already have the DataFrame with the "class" column)
df = pd.read_csv('stock_class.csv')  # Replace 'output_csv_file.csv' with the path to your filtered CSV file

In [25]:
# Define the condition to filter for Class A and Class B
class_condition = (df['class'] == 'A')

# Apply the condition to filter the DataFrame
filtered_df = df[class_condition]

# Save the filtered DataFrame to a new CSV file
filtered_df.to_csv('exports/filtered_class_a_b.csv', index=False)  # Replace 'filtered_class_a_b.csv' with the desired output file path


In [37]:
import pandas as pd

# Load the filtered DataFrame (assuming you already have the DataFrame with the "class" column)
df = pd.read_csv('exports/stock_class.csv')  # Replace 'output_csv_file.csv' with the path to your filtered CSV file
#df = filtered_df

In [38]:
import pandas as pd
import datetime

# Assuming you already have your DataFrame 'df' defined

# Remove the last 3 characters from 'stock_code'
df['stock_code'] = df['stock_code'].str[:-3]

# Rename the columns
df = df.rename(columns={
    'stock_code' : "stock_code",
    'dividendYield': 'rDY',
    'payoutRatio': 'rPR',
    'operatingMargins': 'rOM',
    'CF': 'rCF',
    'P/E': 'rPE',
    'ROE': 'rROE',
    'EPS': 'rEPS',
    'class': 'rClass'
})

# Add today's date to a new column 'rDate'
df['rDate'] = datetime.date.today()

# Multiply 'EPS', 'rOM', 'rDY', and 'rPR' by 100
df['rEPS'] *= 100
df['rOM'] *= 100
df['rDY'] *= 100
df['rPR'] *= 100

# Round all columns to 3 decimal places
rounded_df = df.round(3)

# Specify the output file path and save the rounded DataFrame to CSV
output_file_path = 'exports/stock_class_rounded.csv'
rounded_df.to_csv(output_file_path, index=False)
